### Import dependencies

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

### Import Airline data

In [2]:
messy_flight_df = pd.DataFrame()

# loop through csv files 
for x in range(1,8):
    file = Path(f'Resources/Flights_2022_{x}.csv')
    import_df = pd.read_csv(file,low_memory=False,engine='c')
    messy_flight_df = pd.concat([messy_flight_df,import_df],ignore_index=True)

#### Export combined data into new csv

In [15]:
# USE IF NECESSARY

# export_path = Path('Resources/Flights_2022_Master.csv')
# messy_flight_df.to_csv(export_path,index=False)

#### Re-Import master csv

In [16]:
# USE IF NECESSARY

# messy_import_path = Path('Resources/Flights_2022_Master.csv')
# messy_flight_df = pd.read_csv(messy_import_path,low_memory=False)

### Separate master dataframe into each table

#### Airlines DF

In [4]:
airlines_df_path = Path('Resources/Airlines.csv')
Airlines_df = pd.read_csv(airlines_df_path)

#### Flights DF

In [6]:
# Making new binary column that tracks delay for both arr and dep
messy_flight_df['Delayed'] = np.where((messy_flight_df['ArrDelayMinutes']>0) | (messy_flight_df['DepDelayMinutes']>0), 1, 0)

In [10]:
flights_columns_list = [
    'Flight_Number_Operating_Airline',
    'Operating_Airline ',
    'Origin',
    'Dest',
    'FlightDate',
    'DepTime', # change from military to datetime
    'ArrTime', # change from military to datetime
    'Delayed',
    'Cancelled'
    ]

In [11]:
flight_df_path = Path('Resources/Flights_Table.csv')
Flights_df = messy_flight_df[flights_columns_list]
Flights_df.to_csv(flight_df_path)

#### Delayed Flights DF

In [24]:
delayed_columns_list = [
    'Flight_Number_Operating_Airline',
    'ArrTime',
    'ArrDelayMinutes',
    'DepTime', # change from military to datetime
    'DepDelayMinutes',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

In [25]:
delayed_df_path = Path('Resources/Delayed_Flights_Table.csv')
Delayed_Flights_df = messy_flight_df.loc[(messy_flight_df['ArrDelayMinutes']>0) |
                                         (messy_flight_df['DepDelayMinutes']>0)][delayed_columns_list]
Delayed_Flights_df.to_csv(delayed_df_path)

#### Airport Codes DF

In [14]:
airport_codes_list = [
    'Origin',
    'OriginCityName',
    'OriginState'
]

In [15]:
airport_codes_df_path = Path('Resources/Airport_Codes_Table.csv')
Airport_Codes_df = messy_flight_df[airport_codes_list]
Airport_Codes_df.to_csv(airport_codes_df_path)

#### Cancelled Flights DF

In [16]:
cancelled_flights_list = [
    'Flight_Number_Operating_Airline',
    'CancellationCode'
]

In [17]:
cancelled_df_path = Path('Resources/Cancelled_Flights_Table.csv')
Cancelled_Flights_df = messy_flight_df.loc[messy_flight_df['Cancelled']==1][cancelled_flights_list]
Cancelled_Flights_df.to_csv(cancelled_df_path)

### Format Data

In [3]:
def converter (series):
    # converts the string into a military time format
    series = series.apply(lambda x: f'{(int(x) // 100):02d}:{(int(x) % 100):02d}' if pd.notnull(x) else x)
    # Convert string to correct midnight format
    series = series.apply(lambda x: '00:00' if x == '24:00' else x)
    # Converts values into datetime object
    series = series.apply(lambda x: pd.to_datetime(x,format='%H:%M').strftime('%H:%M') if pd.notnull(x) else x)
    return series

#### Flight Table Cleaning

In [18]:
# Either imports if starting from new session or makes a copy of DF for cleaning
try:
    if Flights_df is not None:
        flight_cleanup_df = Flights_df
except NameError:
    flight_cleanup_df = pd.read_csv(flight_df_path,index_col=0)

In [19]:
# converts the string into a military time format
flight_cleanup_df['ArrTime'] =converter(flight_cleanup_df['ArrTime'])

flight_cleanup_df['DepTime'] =converter(flight_cleanup_df['DepTime'])

C:\Users\lady-\AppData\Local\Temp\ipykernel_15104\2937021756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_cleanup_df['ArrTime'] =converter(flight_cleanup_df['ArrTime'])
C:\Users\lady-\AppData\Local\Temp\ipykernel_15104\2937021756.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_cleanup_df['DepTime'] =converter(flight_cleanup_df['DepTime'])


In [20]:
# Convert FlightDate to datetime object
flight_cleanup_df['FlightDate'] = flight_cleanup_df['FlightDate'].apply(lambda x: pd.to_datetime(x) if pd.notnull(x) else x)

C:\Users\lady-\AppData\Local\Temp\ipykernel_15104\2056941227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_cleanup_df['FlightDate'] = flight_cleanup_df['FlightDate'].apply(lambda x: pd.to_datetime(x) if pd.notnull(x) else x)


In [21]:
# Rename Columns
flight_cleanup_df.rename(columns={
    'Flight_Number_Operating_Airline':'Airline_Flight_Number',
    'Operating_Airline ':'Operating_Airline',
    'Origin':'Origin_Airport',
    'Dest':'Destination_Airport',
    'FlightDate':'Flight_Date',
    'DepTime':'Departure_Time',
    'ArrTime':'Arrival_Time'},inplace=True)

C:\Users\lady-\AppData\Local\Temp\ipykernel_15104\2886356529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_cleanup_df.rename(columns={


In [22]:
flight_cleanup_df

,Airline_Flight_Number,Operating_Airline,Origin_Airport,Destination_Airport,Flight_Date,Departure_Time,Arrival_Time,Delayed,Cancelled
0,1581,DL,FLL,LGA,2022-01-06,NaN,NaN,0,1.0
1,1582,DL,ATL,FLL,2022-01-06,16:27,18:20,0,0.0
2,1582,DL,FLL,ATL,2022-01-06,19:29,21:15,0,0.0
3,1583,DL,FLL,RDU,2022-01-06,10:19,12:12,0,0.0
4,1584,DL,ATL,JAN,2022-01-06,11:13,11:31,0,0.0
...,...,...,...,...,...,...,...,...,...
4078313,3406,YX,ALB,EWR,2022-07-01,15:16,16:30,1,0.0
4078314,3405,YX,AVL,EWR,2022-07-01,12:36,14:28,0,0.0
4078315,3403,YX,ALB,EWR,2022-07-01,11:53,13:33,1,0.0
4078316,3401,YX,BNA,EWR,2022-07-01,14:58,18:43,1,0.0


In [23]:
# Export cleaned Dataframe to csv to use as Table

cleaned_flight_path = Path('Resources/DB_Flights_Table.csv')
flight_cleanup_df.to_csv(cleaned_flight_path)

#### Delayed Flight Table Cleaning

In [27]:
# Either imports if starting from new session or makes a copy of DF for cleaning
try:
    if Delayed_Flights_df is not None:
        delay_cleanup_df = Delayed_Flights_df
except NameError:
    delay_cleanup_df = pd.read_csv(delayed_df_path,index_col=0)

In [28]:
delay_cleanup_df['ArrTime'] = converter(delay_cleanup_df['ArrTime'])
delay_cleanup_df['DepTime'] = converter(delay_cleanup_df['DepTime'])

In [29]:
# Rename Columns
delay_cleanup_df.rename(columns={
    'Flight_Number_Operating_Airline':'Airline_Flight_Number',
    'ArrTime':'Arrival_Time',
    'ArrDelayMinutes':'Arrival_Delayed_Minutes',
    'DepTime':'Departure_Time',
    'DepDelayMinutes':'Departure_Delayed_Minutes',
    'CarrierDelay':'Carrier_Delay_Minutes',
    'WeatherDelay':'Weather_Delay_Minutes',
    'NASDelay':'NAS_Delay_Minutes',
    'SecurityDelay':'Security_Delay_Minutes',
    'LateAircraftDelay':'Late_Aircraft_Delay_Minutes'},
    inplace=True)

In [30]:
delay_cleanup_df

,Airline_Flight_Number,Arrival_Time,Arrival_Delayed_Minutes,Departure_Time,Departure_Delayed_Minutes,Carrier_Delay_Minutes,Weather_Delay_Minutes,NAS_Delay_Minutes,Security_Delay_Minutes,Late_Aircraft_Delay_Minutes
7,1586,14:35,2.0,10:18,18.0,NaN,NaN,NaN,NaN,NaN
8,1587,15:31,0.0,14:22,8.0,NaN,NaN,NaN,NaN,NaN
17,1593,10:22,8.0,06:56,0.0,NaN,NaN,NaN,NaN,NaN
19,1595,12:14,40.0,10:46,51.0,2.0,0.0,0.0,0.0,38.0
21,1597,22:30,7.0,20:58,3.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4078312,3408,18:18,76.0,17:25,55.0,0.0,0.0,76.0,0.0,0.0
4078313,3406,16:30,18.0,15:16,16.0,0.0,0.0,18.0,0.0,0.0
4078315,3403,13:33,20.0,11:53,0.0,0.0,0.0,20.0,0.0,0.0
4078316,3401,18:43,43.0,14:58,28.0,0.0,0.0,43.0,0.0,0.0


In [31]:
delay_cleanup_df.columns

Index(['Airline_Flight_Number', 'Arrival_Time', 'Arrival_Delayed_Minutes',
       'Departure_Time', 'Departure_Delayed_Minutes', 'Carrier_Delay_Minutes',
       'Weather_Delay_Minutes', 'NAS_Delay_Minutes', 'Security_Delay_Minutes',
       'Late_Aircraft_Delay_Minutes'],
      dtype='object')

In [32]:
#Convert Columns from int to Datetime 

delay_cleanup_df['Carrier_Delay_Minutes'] = pd.to_timedelta(delay_cleanup_df['Carrier_Delay_Minutes'],unit='m')

delay_cleanup_df['Weather_Delay_Minutes'] = pd.to_timedelta(delay_cleanup_df['Weather_Delay_Minutes'],unit='m')

delay_cleanup_df['Departure_Delayed_Minutes'] = pd.to_timedelta(delay_cleanup_df['Departure_Delayed_Minutes'],unit='m')

delay_cleanup_df['Arrival_Delayed_Minutes'] = pd.to_timedelta(delay_cleanup_df['Arrival_Delayed_Minutes'],unit='m')

delay_cleanup_df['NAS_Delay_Minutes'] = pd.to_timedelta(delay_cleanup_df['NAS_Delay_Minutes'],unit='m')

delay_cleanup_df['Security_Delay_Minutes'] = pd.to_timedelta(delay_cleanup_df['Security_Delay_Minutes'],unit='m')

delay_cleanup_df['Late_Aircraft_Delay_Minutes'] = pd.to_timedelta(delay_cleanup_df['Late_Aircraft_Delay_Minutes'],unit='m')

delay_cleanup_df

,Airline_Flight_Number,Arrival_Time,Arrival_Delayed_Minutes,Departure_Time,Departure_Delayed_Minutes,Carrier_Delay_Minutes,Weather_Delay_Minutes,NAS_Delay_Minutes,Security_Delay_Minutes,Late_Aircraft_Delay_Minutes
7,1586,14:35,0 days 00:02:00,10:18,0 days 00:18:00,NaT,NaT,NaT,NaT,NaT
8,1587,15:31,0 days 00:00:00,14:22,0 days 00:08:00,NaT,NaT,NaT,NaT,NaT
17,1593,10:22,0 days 00:08:00,06:56,0 days 00:00:00,NaT,NaT,NaT,NaT,NaT
19,1595,12:14,0 days 00:40:00,10:46,0 days 00:51:00,0 days 00:02:00,0 days,0 days 00:00:00,0 days,0 days 00:38:00
21,1597,22:30,0 days 00:07:00,20:58,0 days 00:03:00,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
4078312,3408,18:18,0 days 01:16:00,17:25,0 days 00:55:00,0 days 00:00:00,0 days,0 days 01:16:00,0 days,0 days 00:00:00
4078313,3406,16:30,0 days 00:18:00,15:16,0 days 00:16:00,0 days 00:00:00,0 days,0 days 00:18:00,0 days,0 days 00:00:00
4078315,3403,13:33,0 days 00:20:00,11:53,0 days 00:00:00,0 days 00:00:00,0 days,0 days 00:20:00,0 days,0 days 00:00:00
4078316,3401,18:43,0 days 00:43:00,14:58,0 days 00:28:00,0 days 00:00:00,0 days,0 days 00:43:00,0 days,0 days 00:00:00


In [34]:
# Some times converted to NaT - fill values to 0 days
interval_columns = ['Arrival_Delayed_Minutes', 'Departure_Delayed_Minutes', 'Carrier_Delay_Minutes',
                    'Weather_Delay_Minutes', 'NAS_Delay_Minutes', 'Security_Delay_Minutes',
                    'Late_Aircraft_Delay_Minutes']

delay_cleanup_df[interval_columns] = delay_cleanup_df[interval_columns].fillna(pd.Timedelta(seconds=0))

In [35]:
delay_cleanup_df

,Airline_Flight_Number,Arrival_Time,Arrival_Delayed_Minutes,Departure_Time,Departure_Delayed_Minutes,Carrier_Delay_Minutes,Weather_Delay_Minutes,NAS_Delay_Minutes,Security_Delay_Minutes,Late_Aircraft_Delay_Minutes
7,1586,14:35,0 days 00:02:00,10:18,0 days 00:18:00,0 days 00:00:00,0 days,0 days 00:00:00,0 days,0 days 00:00:00
8,1587,15:31,0 days 00:00:00,14:22,0 days 00:08:00,0 days 00:00:00,0 days,0 days 00:00:00,0 days,0 days 00:00:00
17,1593,10:22,0 days 00:08:00,06:56,0 days 00:00:00,0 days 00:00:00,0 days,0 days 00:00:00,0 days,0 days 00:00:00
19,1595,12:14,0 days 00:40:00,10:46,0 days 00:51:00,0 days 00:02:00,0 days,0 days 00:00:00,0 days,0 days 00:38:00
21,1597,22:30,0 days 00:07:00,20:58,0 days 00:03:00,0 days 00:00:00,0 days,0 days 00:00:00,0 days,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...
4078312,3408,18:18,0 days 01:16:00,17:25,0 days 00:55:00,0 days 00:00:00,0 days,0 days 01:16:00,0 days,0 days 00:00:00
4078313,3406,16:30,0 days 00:18:00,15:16,0 days 00:16:00,0 days 00:00:00,0 days,0 days 00:18:00,0 days,0 days 00:00:00
4078315,3403,13:33,0 days 00:20:00,11:53,0 days 00:00:00,0 days 00:00:00,0 days,0 days 00:20:00,0 days,0 days 00:00:00
4078316,3401,18:43,0 days 00:43:00,14:58,0 days 00:28:00,0 days 00:00:00,0 days,0 days 00:43:00,0 days,0 days 00:00:00


In [36]:
# Export cleaned Dataframe to csv to use as Table

cleaned_delay_path = Path('Resources/DB_Delayed_Flights_Table.csv')
delay_cleanup_df.to_csv(cleaned_delay_path)

#### Airport Codes Table Cleaning

In [40]:
# Either imports if starting from new session or makes a copy of DF for cleaning
try:
    if Airport_Codes_df is not None:
        airport_codes_cleanup_df = Airport_Codes_df
except NameError:
    airport_codes_cleanup_df = pd.read_csv(airport_codes_df_path,index_col=0)

In [41]:
# get only the Unique Origin Airport Codes
airport_codes_cleanup_df.drop_duplicates(subset=['Origin'],inplace=True)

In [42]:
# Split to create columns for Table
airport_codes_cleanup_df[['Airport_City','Airport_State']] = airport_codes_cleanup_df['OriginCityName'].str.split(', ',expand=True)

In [43]:
# Remove unnecesary Columns
airport_codes_cleanup_df.drop(columns=['OriginState','OriginCityName'],inplace=True)

In [44]:
# Clean the index
airport_codes_cleanup_df.reset_index(drop=True,inplace=True)

In [45]:
# Rename Columns
airport_codes_cleanup_df.rename(columns={'Origin':'Airport_Code'},inplace=True)

In [46]:
airport_codes_cleanup_df

,Airport_Code,Airport_City,Airport_State
0,FLL,Fort Lauderdale,FL
1,ATL,Atlanta,GA
2,JAN,Jackson/Vicksburg,MS
3,RIC,Richmond,VA
4,MSP,Minneapolis,MN
...,...,...,...
370,ACK,Nantucket,MA
371,GST,Gustavus,AK
372,HYA,Hyannis,MA
373,MVY,Martha's Vineyard,MA


In [47]:
# Export cleaned Dataframe to csv to use as Table

cleaned_airport_codes_path = Path('Resources/DB_Airport_Codes.csv')
airport_codes_cleanup_df.to_csv(cleaned_airport_codes_path)

#### Airline Table Cleaning

In [48]:
# Either imports if starting from new session or makes a copy of DF for cleaning
try:
    if Airlines_df is not None:
        airlines_cleanup_df = Airlines_df
except NameError:
    airlines_cleanup_df = pd.read_csv(airlines_df_path)

In [49]:
# Rename columns

airlines_cleanup_df.rename(columns={
    'Code':'Airline_Code',
    'Description':'Airline_Name'},inplace=True)

In [50]:
airlines_cleanup_df

,Airline_Code,Airline_Name
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.
...,...,...
1566,ZW,Air Wisconsin Airlines Corp
1567,ZX,Air Georgian
1568,ZX (1),Airbc Ltd.
1569,ZY,Atlantic Gulf Airlines


In [51]:
# Export cleaned Dataframe to csv to use as Table

cleaned_airlines_path = Path('Resources/DB_Airline_Table.csv')
airlines_cleanup_df.to_csv(cleaned_airlines_path)

#### Cancelled Flights Table Cleaning

In [52]:
# Import Table Data
try:
    if Cancelled_Flights_df is not None:
        cancelled_cleanup_df = Cancelled_Flights_df
except NameError:
    cancelled_cleanup_df = pd.read_csv(cancelled_df_path,index_col=0)

In [53]:
cancelled_cleanup_df.rename(columns={'Flight_Number_Operating_Airline':'Airline_Flight_Number',
                                     'CancellationCode':'Cancellation_Code'},
                            inplace=True)

In [54]:
cancelled_cleanup_df

,Airline_Flight_Number,Cancellation_Code
0,1581,A
30,1605,B
75,1650,B
148,1722,B
157,1729,B
...,...,...
4078270,3463,C
4078285,3443,C
4078290,3434,C
4078301,3421,C


In [55]:
# Export cleaned Dataframe to csv to use as Table

cancelled_flight_path = Path('Resources/DB_Cancelled_Flights_Table.csv')
cancelled_cleanup_df.to_csv(cancelled_flight_path)